In [ ]:
!pip install deap update_checker tqdm stopit tpot

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.4/87.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 MB 7.4 MB/s eta 0:00:00
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11938 sha256=53d1eb74b96c87cfda0d8e5716f873d3e4e0d81814afa5ff54a8485f38fb7c44
  Stored in directory: /root/.cache/pip/wheels/af/f9/87/bf5b3d565c2a007b4dae9d8142dccc85a9f164e517062dd519
Successfully built stopit
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.2
    Uninstalling scikit-learn-1.3.2:
      Successfully uninstalled scikit-learn-1.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.

In [ ]:
import pandas as pd
import numpy as np
from tpot import TPOTRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer


# Load the dataset
df = pd.read_csv("combined_regression_normal.csv") ##dataset without scraping_time and ad_time, leads to error

df = df.dropna(subset=['Price'])  # Remove rows with missing prices

# Separate features and target
X = df.drop(['Price'], axis=1)
y = df['Price']

# One-Hot Encode categorical variables
categorical_cols = X.select_dtypes(include=['object']).columns
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_cols)],
    remainder='passthrough')

X = preprocessor.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


In [ ]:
# initializing tpot with parameters
#max_time_mins=None (per default) - here max time 4h
#config_dic due to one hot encoding
tpot = TPOTRegressor(generations=5, population_size=50,
                     cv=5, config_dict='TPOT sparse', verbosity=2, n_jobs=-1, max_time_mins=240,
                     periodic_checkpoint_folder='/content/results')

In [ ]:
#starting the training
tpot.fit(X_train, y_train)

Optimization Progress:   0%|          | 0/5 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: -3135226901462.7183

Generation 2 - Current best internal CV score: -3135226901462.7183

Best pipeline: RandomForestRegressor(input_matrix, bootstrap=True, max_features=0.7000000000000001, min_samples_leaf=6, min_samples_split=20, n_estimators=100)


TPOTRegressor(config_dict='TPOT sparse', cv=2, generations=2, max_time_mins=2,
              n_jobs=-1, periodic_checkpoint_folder='/content/results',
              population_size=5, verbosity=2)

In [ ]:
import sklearn.metrics as skm

y_pred = tpot.predict(X_test)

print("RESULTS OF BEST MODEL:\n")

print(f"R2:                         {skm.r2_score(y_test, y_pred)}")
print(f"Mean Squared Error:         {skm.mean_squared_error(y_test, y_pred)}")
print(f"Root Mean Squared Error:    {skm.mean_squared_error(y_test, y_pred, squared=False)}")
print(f"Mean Absolute Error:        {skm.mean_absolute_error(y_test, y_pred)}")
print(f"Explained Variance:         {skm.explained_variance_score(y_test, y_pred)}")
print(f"Max Error:                  {skm.max_error(y_test, y_pred)}")
print(f"Median Absolute Error       {skm.median_absolute_error(y_test, y_pred)}")


RESULTS OF BEST MODEL:

R2:                         0.46008692573975585
Mean Squared Error:         2953343869441.299
Root Mean Squared Error:    1718529.5660655068
Mean Absolute Error:        954269.2799393244
Explained Variance:         0.4602554415852057
Max Error:                  38256056.77042082
Median Absolute Error       571874.9239247751


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [ ]:
from google.colab import files
# output of values to dictionary
metrics_dict = {
    'Metric': [
        'R2',
        'Mean Squared Error',
        'Root Mean Squared Error',
        'Mean Absolute Error',
        'Explained Variance',
        'Max Error',
        'Median Absolute Error'
    ],
    'Value': [
        skm.r2_score(y_test, y_pred),
        skm.mean_squared_error(y_test, y_pred),
        skm.mean_squared_error(y_test, y_pred, squared=False),
        skm.mean_absolute_error(y_test, y_pred),
        skm.explained_variance_score(y_test, y_pred),
        skm.max_error(y_test, y_pred),
        skm.median_absolute_error(y_test, y_pred)
    ]
}

#to df
metrics_df = pd.DataFrame(metrics_dict)


print(metrics_df)

metrics_df.to_csv('tpot_regression_metrics_normal.csv', index=False)
files.download('tpot_regression_metrics_normal.csv')

                    Metric         Value
0                       R2  4.600869e-01
1       Mean Squared Error  2.953344e+12
2  Root Mean Squared Error  1.718530e+06
3      Mean Absolute Error  9.542693e+05
4       Explained Variance  4.602554e-01
5                Max Error  3.825606e+07
6    Median Absolute Error  5.718749e+05


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
tpot.export('tpot_regression_model_normal')